In [5]:
import numpy as np
import torch.nn as nn
import torch

In [18]:
def conv2d(input, kernel, bias, stride, padding):
    if padding > 0:
        input = np.pad(input, 
                        ((0, 0), (0, 0), (padding, padding), (padding, padding)), 
                        "constant", 
                        constant_values=0)
    batch_size, in_channel, x_h, x_w=  input.shape
    out_channel, in_channel, k_h, k_w = kernel.shape
    
    y_h = (x_h - k_h) // stride + 1
    y_w = (x_w - k_w) // stride + 1
    output = np.zeros((batch_size, out_channel, y_h, y_w))  

    for bs in range(batch_size):
        for oc in range(out_channel):
            output[bs][oc] += bias[oc]
            for ic in range(in_channel):
                for i in range(0, x_h-k_h+1, stride):
                    for j in range(0, x_w-k_w+1, stride):
                        region = input[bs, ic, i:i+k_h, j:j+k_w]
                        
                        output[bs, oc, int(i//stride), int(j //)] += np.sum(region * kernel[oc, ic])

    return output

In [19]:
x = np.random.randn(128, 3, 5,5)
kernel = np.random.randn(5, 3, 3, 3)
bias = np.random.randn(5)
y = conv2d(x, kernel, bias, 1, 0)
print(y.shape)

(128, 5, 3, 3)


In [17]:
class RNN(nn.Module):
    def __init__(self, in_feature, out_feature, seq_len):
        """
        params:
            in_feature：输入维度
            out_feature：输出维度
            seq_len: 序列长度
        """
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(in_feature, out_feature, 2)

        # 每个时刻适用不同的线性模型
        self.out = nn.Sequential()
        for i in range(seq_len):
            self.out.add_module(name="t" + str(i), 
                                module = nn.Linear(out_feature, 1))
            
    
    def forward(self, x):
        seq_len, batch_szie, in_feature = x.shape

        x_1, _ = self.lstm(x)   
        a, b, c = x_1.shape  # [seq_len, batch_size, out_feature]

        # 对每个时刻进行转换
        outputs = torch.zeros((seq_len, batch_szie, 1))
        for i in range(seq_len):
            outputs[i,:,:] = self.out[i](x_1[i, :, :])
            
        return outputs

rnn = RNN(in_feature=8, out_feature=16, seq_len=14)

In [18]:
x = np.random.randn(14, 1, 8)
x = torch.from_numpy(x).to(torch.float32)

In [20]:
y = rnn(x)
y.shape

torch.Size([14, 1, 1])

In [ ]:
def conv2d(X, K, stride, bias, padding):

    X = np.pad(array=X, 
                pad_width=((0, 0), (0, 0), (padding, padding), (padding, padding)),
                mod='constant', 
                constant_values=0)

    batch_size, x_in, x_h, x_w = X.shape
    out_channel, in_channel, k_h, k_w = K.shape
    assert x_in == in_channel, "维度不匹配"
    
    y_h = (x_h - k_h) // stride + 1
    y_w = (x_w - k_w) // stride + 1
    
    Y = np.zeros((batch_size, out_channel, y_h, y_w))

    if bias is None:
        bias = np.zeros(( out_channel))

    for bs in range(batch_size):
        for oc in range(out_channel):
            Y[:, oc, :, :] += bias[oc]
            for ic in range(in_channel):
                for i in range(0, x_h-k_h+1, stride):
                    for j in range(0, x_w-k_w+1, stride):
                        region = X[bs, ic, i:i+k_h, j:j+k_w]
                        kernel = K[oc, ic]
                        Y[bs, oc, int(i//stride), int(j//stride)] += np.sum(region * kernel)

    return Y

In [2]:
from typing import List
def matrix_matual(A, B) -> List[List[int]]:
    m, n = len(A), len(A[0])
    t, q = len(B), len(B[0])

    assert n == t, "维度不匹配"
    
    res = [[0] * q for _ in range(m)]

    for i in range(0, m):
        for j in range(0, q):
            for k in range(0, n):
                res[i][j] += A[i][k] * B[k][j]

    return res

In [8]:
import numpy as np
A = np.random.randint(0, 20, (3, 8))
B = np.random.randint(0, 70, (8, 5))

res_np = np.matmul(A, B)
res_my = matrix_matual(A.tolist(), B.tolist())

res_np, res_my

(array([[2187,  854, 2147, 1793, 1988],
        [4190, 2372, 3833, 3067, 3638],
        [3383, 1568, 2542, 2316, 2613]]),
 [[2187, 854, 2147, 1793, 1988],
  [4190, 2372, 3833, 3067, 3638],
  [3383, 1568, 2542, 2316, 2613]])

In [17]:
import math
math.ceil(5/2)

3

In [ ]:
class Module:
    def __init__(self):
        pass

    def __call__(self, *arg, **kwarg):
        return self.forward(*arg, **kwarg)

    def forward(self, *arg, **kwarg):
        pass
    
    def backwarg(self, *arg, **kwarg):
        pass
        

In [ ]:
class Conv2d(Module):
    def __init__(self, out_channel, in_channel, k_w, k_h, stride, padding):
        self.out_channel = out_channel
        self.in_channel = in_channel
        self.k_h = k_h
        self.k_w = k_w
        self.stride = stride
        self.padding = padding

        self._init_params()
        

    def _init_params(self):
        self.K = np.random.normal((self.out_channel, self.in_channel, self.k_h, self.k_w))
        self.bias = np.random.random((self.out_channel))


    def forward(self, X):
        return self._conv2d(X, self.K, self.bias, self.stride, self.padding)
        
    
    def _conv2d(self, X, K, bias, stride, padding):
        X = np.pad(array=X, 
                    pad_width=((0, 0), (0, 0), (padding, padding), (padding, padding)),
                    mod='constant', 
                    constant_values=0)

        batch_size, x_in, x_h, x_w = X.shape
        out_channel, in_channel, k_h, k_w = K.shape
        assert x_in == in_channel, "维度不匹配"
        
        y_h = (x_h - k_h) // stride + 1
        y_w = (x_w - k_w) // stride + 1
        
        Y = np.zeros((batch_size, out_channel, y_h, y_w))

        if bias is None:
            bias = np.zeros(( out_channel))

        for bs in range(batch_size):
            for oc in range(out_channel):
                Y[:, oc, :, :] += bias[oc]
                for ic in range(in_channel):
                    for i in range(0, x_h-k_h+1, stride):
                        for j in range(0, x_w-k_w+1, stride):
                            region = X[bs, ic, i:i+k_h, j:j+k_w]
                            kernel = K[oc, ic]
                            Y[bs, oc, int(i//stride), int(j//stride)] += np.sum(region * kernel)

        return Y

In [5]:
def rnn_forward(input_, init_h, W_hx, W_hh, b):
    """ input_ :[batch_size, seq_len, input_size]
        init_h: 初始 状态向量 h [batch_size, output_size]
        W_hx: 参数矩阵    [input_size, output_size]
        W_hh: 参数矩阵    [output_size, output_size]
        b: 偏置参数
    """
    output_size, _ = W_hh.shape
    batch_size, seq_len, input_size = input_.shape
    outputs = np.zeros((batch_size, seq_len, output_size))

    prev_h = init_h
    for i in range(0, seq_len):
        cur_x = input_[:, i:, :]
        cur_h = np.tanh(W_hh @  prev_h + W_hx @ cur_x + b)
        
        outputs[:, :, :] = cur_h
        prev_h = cur_h

    return outputs, cur_h.reshape(1, batch_size, output_size)

In [1]:
import numpy as np


class RNNCell:
    def __init__(self, input_size, hidden_size, bias, nonlinearity='tanh'):
        """
        parameters:
            input_size: The number of expected features in the input x
            hidden_size: The number of features in the hidden state h
            bias: If False, then the layer does not use bias weights b. Default: True
            nonlinearity: The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
        """
        # init_param 
        self.W_hh = np.random.normal(loc=0, scale=np.sqrt(1/hidden_size))
        self.W_hx = np.random.normal(loc=0, scale=np.sqrt(2/(input_size+hidden_size)))
        self.bias = bias
        if self.bias:
            self.b = np.zeros((hidden_size))
        self.nonlinearity = nonlinearity


    def forward(self, x, prev_h):
        """input_ :[batch_size, input_size]
        """
        a = prev_h @ self.W_hh + x @ self.W_hx
        if self.bias:
            a += self.b
        if self.nonlinearity == "tanh":
            a = np.tanh(a)
        elif self.nonlinearity == "relu":
            a = self.relu(a)
        return a


    def __call__(self, *arg):
        return self.forward(*arg)     

    def backward(self, pre_grad):
        pass

    def relu(self, x):
        mask = x < 0
        x[mask] = 0
        return x
        

In [ ]:
class RNN:
    def __init__(self, input_size, hidden_size, bias, nonlinearity):
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = RNNCell(self.input_size, self.hidden_size, bias, nonlinearity)
        
    def forward(self, input_, prev_h):
        """
        parameters:
            input_ :[batch_size, seq_len, input_size]
            prev_h: [batch_size, seq_len, hidden_size]
        return :
            outputs: [batch_size, seq_len, hidden_size]
            cur_h: [1, batch_size, hidden_size]
        """
        batch_size, seq_len, input_size = input_.shape

        outputs = np.zeros((batch_size, seq_len, self.hidden_size))

        for i in range(0, seq_len):
            cur_x = input_[:, i, :]
            cur_h = self.cell(cur_x, prev_h)
            
            outputs[:, i, :] = cur_h
            prev_h = cur_h

        return outputs, cur_h.reshape(1, batch_size, self.hidden_size)  

    def __call__(self, *arg):
        return self.forward(*arg)

    def backward(self, pre_grad):
        pass      

In [4]:
x = np.random.randint(-10, 10, size=(4, 4))
x

array([[-1,  6, -6,  5],
       [ 8,  9, -9,  8],
       [-2,  7,  9,  5],
       [-5, -8, -4, -5]])

In [7]:
mask = x < 0
mask

array([[ True, False,  True, False],
       [False, False,  True, False],
       [ True, False, False, False],
       [ True,  True,  True,  True]])

In [9]:
x[mask] = 0 
x

array([[0, 6, 0, 5],
       [8, 9, 0, 8],
       [0, 7, 9, 5],
       [0, 0, 0, 0]])

In [1]:
import numpy as np

def conv2d(X, K, b):
    """
    X: 输入特征图, shape:[x_w, x_h]
    K: 卷积核, shape:[k_w, k_h]
    """
    x_w, x_h = X.shape
    k_w, k_h = K.shape
    
    # 输出特征图大小
    y_w, y_h = x_w - k_w + 1, x_h-k_h+1
    Y = np.zeros((y_w, y_h))
    
    # 计算输出特征图的每个值
    for i in range(0, y_h):
        for j in range(0, y_w):
            conv_region = X[i:i+k_h, j:j+k_w]
            Y[i, j] = np.sum(conv_region * K) + b

    return Y

In [ ]:
def conv2d(X, K, b, stride, padding):
    """
    X: 输入特征图, shape:[x_w, x_h]
    K: 卷积核, shape:[k_w, k_h]
    """
    # 填充
    if padding > 0:
        X  =np.pad(array=X, 
                    pad_width=((padding, padding), (padding, padding)),
                    mode="constant",
                    condtant_values = 0)
    
    x_h, x_w = X.shape
    k_h, k_w = K.shape

    y_h = (x_h - k_h) // stride + 1
    y_w = (x_w - k_w) // stride + 1
    
    Y = np.zeros((y_h, y_w))
    
    for i in range(0, x_h-y_w+1, stride):
        for j in range(x_w-k_w+1, stride):
            conv_region = X[i:i+k_h, j:j+k_w]
            Y[i//stride][j//stride]  = np.sum(conv_region * K) + b

    return Y

In [ ]:
def conv_2d(X, K, bias, stride, paddind):
    batch_size, in_channel, x_h, x_w = X.shape
    out_channel, in_channel, k_h, k_w = K.shape
    
    y_h = (x_h-k_h) // stride + 1
    y_w = (x_w-k_w) // stride + 1

    Y = np.zeros((batch_size, out_channel, y_h, y_w))
    for b_s in range(0, batch_size):
        for o_c in range(0, out_channel):
            Y[b_s, o_c] += bias[oc]
            for i_c in range(0, in_channel):
                for i in range(0, x_h-k_h+1, stride):
                    for j in range(0, x_w-k_w+1, stride):
                        conv_region = X[b_s, i_c, i:i+k_h, j:j+k_h]
                        Y[b_s, o_c, i//stride, j//stride] += np.sum(conv_region * K[o_c, i_c])
            

    return Y

In [ ]:
class Conv2d:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kenel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias

        self.kernel = np.random.normal((self.out_channels, self.in_channels, *self.kernel_size))
        self.b = np.zeros((self.out_channels)) if self.bias else None

    def forward(self, X):
        return self._conv2d(X, self.kernel, self.b, self.stride, self.padding)

    def backward(self, pre_grad):
        pass

    def __call__(self, *arg):
        return self.forward(*arg)


    def _conv2d(self, X, K, b, stride, padding):
        pass

In [ ]:
def _conv2d(self, X, K, b, stride, padding):
    """
    param:
        X: 输入特征图 
            type: np.array
            shape: [batch_size, in_channels, x_h, x_w]
        K: 卷积核 
            type: np.array
            shape: [out_channels, in_channels, k_h, k_w]
        stride: 卷积步长
        padding: padding
    """
    if padding > 0:
        X  =np.pad(array=X, 
                    pad_width=((0, 0), (0, 0),(padding, padding), (padding, padding)),
                    mode="constant",
                    condtant_values = 0)

        batch_size, in_channels_x, x_h, x_w = X.shape
        out_channels, in_channels_k, k_h, k_w = K.shape

        assert in_channels_x == in_channels_k, "输入特征图和卷积核维度不匹配"

        y_h = (x_h - k_h) // stride + 1
        y_w = (x_w - k_w) // stride + 1

        if b is None:
            b = np.zeros((out_channels))
        
        Y = np.zeros((batch_size, out_channels, y_h, y_w))

        for b_s in range(0, batch_size):
            for o_c in range(0, out_channels):
                Y[b_s, o_c] += b[o_c]
                for i_c in range(0, in_channels_k):
                    for i in range(0,x_h-k_h+1 , stride):
                        for j in range(0,x_w-k_w+1 ,stride):
                            conv_region = X[b_s, i_c, i:i+k_h, j:j+k_w]
                            Y[b_s, o_c, i//stride, j//stride] = np.sum(conv_region * K[o_c, i_c])
        return Y

In [4]:

a = np.random.randint(0, 3, (3, 4, 5,2))

In [3]:
a

array([[[[2, 0, 0, 1, 1],
         [0, 1, 2, 0, 2],
         [0, 0, 0, 1, 2],
         [0, 0, 0, 2, 0],
         [1, 0, 0, 2, 2]],

        [[1, 1, 0, 0, 2],
         [1, 1, 0, 0, 1],
         [0, 2, 0, 2, 1],
         [1, 0, 0, 1, 2],
         [2, 0, 0, 0, 0]],

        [[1, 0, 1, 0, 2],
         [1, 0, 2, 2, 1],
         [1, 0, 1, 1, 1],
         [0, 2, 2, 0, 0],
         [2, 1, 0, 2, 2]],

        [[2, 0, 2, 2, 2],
         [0, 1, 2, 2, 1],
         [1, 2, 1, 1, 0],
         [0, 0, 2, 0, 1],
         [0, 2, 1, 0, 1]]],


       [[[0, 1, 2, 2, 0],
         [1, 1, 1, 1, 2],
         [0, 2, 2, 2, 0],
         [2, 0, 1, 1, 1],
         [2, 0, 1, 1, 0]],

        [[2, 1, 2, 1, 0],
         [2, 2, 0, 1, 0],
         [1, 0, 1, 2, 2],
         [2, 1, 0, 1, 2],
         [2, 0, 1, 1, 1]],

        [[0, 2, 2, 2, 1],
         [0, 1, 1, 0, 1],
         [2, 2, 1, 1, 2],
         [0, 2, 0, 0, 0],
         [1, 0, 0, 2, 1]],

        [[1, 0, 2, 0, 1],
         [1, 1, 1, 2, 0],
         [0, 0, 2, 2, 

In [6]:
a[2, 3, :, :]

array([[0, 1],
       [1, 1],
       [2, 1],
       [1, 1],
       [2, 2]])